In [146]:
import pandas as pd
import numpy as np
from shapely.geometry import Point
import geopandas as gpd

In [147]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c
def distance_meters(mercator_start, mercator_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return mercator_start.distance(mercator_finish)

In [148]:
def monumentos():
    monuments = pd.read_csv("../data/monumentos-edificios.csv", delimiter=";")
    monuments['NUM']= monuments['NUM'].astype(str).replace("nan"," SN")
    monuments["Dirección"] = monuments["CLASE-VIAL"] + " " + monuments["NOMBRE-VIA"] + " Nº" + monuments["NUM"]
    return monuments.to_json("../data/monuments.json", orient='records')
def stations():
    stations = pd.read_json("../data/bicimad_stations.json", orient='records')
    geometry_coordinates = stations["geometry_coordinates"].str.split(expand=True)
    geometry_coordinates.columns = ['LONGITUD', 'LATITUD']
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace("[","")
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace(",","")
    geometry_coordinates['LATITUD'] = geometry_coordinates['LATITUD'].str.replace("]","")
    stations = pd.concat([stations, geometry_coordinates], axis=1)
    return stations.to_json("../data/stations.json", orient='records')

In [149]:
monumentos()
stations()

monuments = pd.read_json("../data/monuments.json", orient='records')
stations = pd.read_json("../data/stations.json", orient='records')

/Users/albertofernandez/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # Remove the CWD from sys.path while we load stuff.
/Users/albertofernandez/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if sys.path[0] == '':


In [150]:
monuments["Mercator_start"] = monuments.apply(lambda x: to_mercator(x['LATITUD'],x['LONGITUD']),axis=1)
stations["Mercator_finish"] = stations.apply(lambda x: to_mercator(x['LATITUD'],x['LONGITUD']),axis=1)

In [151]:
stations_clean = stations[["name","address","LONGITUD","LATITUD","Mercator_finish"]]
stations_clean = stations_clean.rename(columns={"name": "BiciMAD station", "address": "Station location", "LONGITUD": "Long_start", "LATITUD": "Lat_start"}, errors="raise")

In [152]:
monuments["Type of place"]= "Edificios de carácter monumental"
monuments_clean = monuments[["NOMBRE","Type of place","Dirección","LONGITUD","LATITUD","Mercator_start"]]
monuments_clean = monuments_clean.rename(columns={"NOMBRE": "Place of interest", "Dirección": "Place address", "LONGITUD": "Long_finish", "LATITUD": "Lat_finish"}, errors="raise")

In [153]:
df_sumary = pd.merge(monuments_clean,stations_clean, how="cross")

In [162]:
df_sumary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55968 entries, 0 to 55967
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Place of interest  55968 non-null  object 
 1   Type of place      55968 non-null  object 
 2   Place address      55968 non-null  object 
 3   Long_finish        55440 non-null  float64
 4   Lat_finish         55440 non-null  float64
 5   Mercator_start     55968 non-null  object 
 6   BiciMAD station    55968 non-null  object 
 7   Station location   55968 non-null  object 
 8   Long_start         55968 non-null  float64
 9   Lat_start          55968 non-null  float64
 10  Mercator_finish    55968 non-null  object 
 11  Distance           55968 non-null  float64
dtypes: float64(5), object(7)
memory usage: 5.6+ MB


In [155]:
prueba= df_sumary.head()

In [160]:
df_sumary["Distance"] = df_sumary.apply(lambda x: distance_meters(x['Mercator_start'],x['Mercator_finish']),axis=1)


In [161]:
df_sumary

,Place of interest,Type of place,Place address,Long_finish,Lat_finish,Mercator_start,BiciMAD station,Station location,Long_start,Lat_start,Mercator_finish,Distance
0,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA Nº3,-3.700924,40.419716,POINT (4499502.203184392 -412271.7922049629),Puerta del Sol A,Puerta del Sol nº 1,-3.701834,40.417214,POINT (4499223.648366921 -412373.2851572837),296.468558
1,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA Nº3,-3.700924,40.419716,POINT (4499502.203184392 -412271.7922049629),Puerta del Sol B,Puerta del Sol nº 1,-3.701603,40.417313,POINT (4499234.670122574 -412347.498528782),278.038463
2,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA Nº3,-3.700924,40.419716,POINT (4499502.203184392 -412271.7922049629),Miguel Moya,Calle Miguel Moya nº 1,-3.705842,40.420589,POINT (4499599.340516399 -412820.3206106317),557.062899
3,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA Nº3,-3.700924,40.419716,POINT (4499502.203184392 -412271.7922049629),Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,-3.706917,40.430294,POINT (4500679.707306497 -412940.3068148262),1354.041263
4,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA Nº3,-3.700924,40.419716,POINT (4499502.203184392 -412271.7922049629),Malasaña,Calle Manuela Malasaña nº 5,-3.702587,40.428552,POINT (4500485.866677178 -412457.3286497012),1001.008311
...,...,...,...,...,...,...,...,...,...,...,...,...
55963,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO Nº SN,-3.761360,40.411760,POINT (4498616.578388661 -419013.7871729974),Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,-3.726990,40.443750,POINT (4502177.655770458 -415179.5220676512),5232.863558
55964,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO Nº SN,-3.761360,40.411760,POINT (4498616.578388661 -419013.7871729974),Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,-3.726930,40.443420,POINT (4502140.920338497 -415172.8287427756),5212.863689
55965,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO Nº SN,-3.761360,40.411760,POINT (4498616.578388661 -419013.7871729974),Facultad Biología,Calle José Antonio Novais frente al nº 12,-3.727295,40.448332,POINT (4502687.743941171 -415213.4906984274),5569.258681
55966,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO Nº SN,-3.761360,40.411760,POINT (4498616.578388661 -419013.7871729974),Facultad Derecho,Avenida Complutense nº 23,-3.729370,40.451090,POINT (4502994.74083288 -415445.024322674),5648.395761
